In [1]:
# import the necessary packages
from sklearn.neighbors import KNeighborsClassifier
from skimage import exposure
from skimage import feature
from imutils import paths
import argparse
import imutils
import cv2
import os
from tqdm.notebook import trange, tqdm

location = 'C:\\Users\\ak19919\\Downloads\\Github\\Analytics-Vidya\\janata-hack_computer_vision_hackathon\\vehicles'

data = []
labels = []

In [2]:
DIR_TRAIN = location + '\\train_labelled'
DIR_TEST = location + '\\test_labelled'

In [3]:
# loop over the image paths in the training set
for imagePath in tqdm(os.listdir(DIR_TRAIN)):
    # extract the label of the vehicle
    label = imagePath.split('-')[0]
    path = os.path.join(DIR_TRAIN, imagePath)
    # load the image, convert it to grayscale, and detect edges
    image = cv2.imread(path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edged = imutils.auto_canny(gray)

    # find contours in the edge map, keeping only the largest one which
    # is presmumed to be the car logo
    cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key = cv2.contourArea)

    # extract the logo of the car and resize it to a canonical width
    # and height
    (x, y, w, h) = cv2.boundingRect(c)
    logo = gray[y:y + h, x:x + w]
    logo = cv2.resize(logo, (224, 224))

    # extract Histogram of Oriented Gradients from the logo
    H = feature.hog(logo, orientations = 9, pixels_per_cell = (10, 10),
        cells_per_block = (2, 2), transform_sqrt = True, block_norm = "L1")

    # update the data and labels
    data.append(H)
    labels.append(label)

In [4]:
# "train" the nearest neighbors classifier
print("[INFO] training classifier...")
model = KNeighborsClassifier(n_neighbors = 1)
model.fit(data, labels)
print("[INFO] evaluating...")

[INFO] training classifier...
[INFO] evaluating...


In [5]:
for imagePath in tqdm(os.listdir(DIR_TEST)):
    # extract the label of the vehicle
    i = int(imagePath.split('-')[1].split('.')[0])
    path = os.path.join(DIR_TEST, imagePath)
    # load the image, convert it to grayscale, and detect edges
    image = cv2.imread(path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    logo = cv2.resize(gray, (224, 224))

    # extract Histogram of Oriented Gradients from the test image and
    # predict the make of the car
    (H, hogImage) = feature.hog(logo, orientations = 9, pixels_per_cell = (10, 10),
        cells_per_block = (2, 2), transform_sqrt = True, block_norm = "L1", visualize = True)
    pred = model.predict(H.reshape(1, -1))[0]

    # visualize the HOG image
    hogImage = exposure.rescale_intensity(hogImage, out_range = (0, 255))
    hogImage = hogImage.astype("uint8")
    cv2.imshow("HOG Image #{}".format(i + 1), hogImage)

    # draw the prediction on the test image and display it
    cv2.putText(image, pred.title(), (10, 35), cv2.FONT_HERSHEY_SIMPLEX, 1.0,
        (0, 255, 0), 3)
    cv2.imshow("Test Image #{}".format(i + 1), image)
    cv2.waitKey(0)